图片的几何变换包括：图片缩放、图片剪切、图片位移、图片镜像和图片仿射变换。

位移、旋转和缩放都是通过矩阵的仿射变换来实现的。

本质是对图片数据进行矩阵运算操作。

## 图片缩放
<img src='images/图片缩放.jpg' width=50%>

In [1]:
# 1.load 2.info 3.resize 4.check
import cv2
img = cv2.imread('image0.jpg',1)
imgInfo = img.shape
print(imgInfo)
height = imgInfo[0] # 图片的行对应高度
width = imgInfo[1] # 图片的列对应宽度
mode = imgInfo[2]
# 缩放分为等比例缩放和非等比例缩放
# 等比例缩放就是原图像的长宽乘以相同的系数
dstHeight = int(height*0.5)
dstWidth = int(width*0.5)

dst = cv2.resize(img,(dstWidth,dstHeight))
cv2.imshow('image',dst)
cv2.waitKey(0)

(547, 730, 3)


-1

OpenCV提供了四种常见的缩放方法：最近邻插值、双线性插值、像素关系重采样和立方插值，默认使用双线性插值。

[图像放缩中最近邻插值和双线性插值的基本原理](https://blog.csdn.net/Andrew659/article/details/4818988)  
[三十分钟理解：线性插值，双线性插值算法](https://blog.csdn.net/xbinworld/article/details/65660665)

**最近邻插值**：  
现在对图像进行缩放，源图像src:10\*20，目标图像dst:5\*10，目标图像的每个点都来自源图像，比如目标图像的(1,2)点用源图像的(2,4)点表示。  
目标图像上的每个点的位置都可以用(h,w)表示，位置(h,w)对应于源图像的点(newH,newW)，计算公式如下：
* newH = h\*(src 行 / 目标 行)  , newH = 1\*(10 / 5） = 2
* newW = w\*(src 列 / 目标 列)  , newW = 2\*(20 / 10）= 4

比如目标图像的点(1,2)对应源图像的点(2,4)，计算出来的结果如果是小数，则进行四舍五入取最近的点，这种方法就是最近邻插值法。  
双线性插值不是用四舍五入法。


**双线性插值**：  
<img src="images/双线性插值.jpg" width="30%">
上图中四个角分别表示四个像素点的坐标，通过线性插值的计算方法得到目标图像上的点对应于源图像上的点为(15.2, 22.3)，这里先把该点在水平和竖直方向上做投影，得到A1,A2,B1,B2。  
A1的纵坐标为15.2，表示A1这个点的值等于0.8乘以点(15,22)加上0.2乘以(16,22)，通过类似的方法也可以得到A2的值。  
B1的横坐标为22.3，表示B1这个点的值等于0.7乘以点(15,22)加上0.3乘以(15,23)，通过类似的方法也可以得到B2的值。  
**注意：A1离(15,22)的距离只有0.2，说明该点的作用应该大一些，所以要用0.8乘以该点，即距离近的点权重大。**  
计算出A1,A2，最终点等于：A1\*70% + A2\*30%。  
计算出B1,B2，最终点等于：B1\*80% + B2\*20%。

用简单的方法表示：  
对于一个目标像素，通过计算得到在原图像上的的坐标为(i+u,j+v) (其中i、j均为坐标的整数部分，u、v为坐标的小数部分)，则这个像素的值 f(i+u,j+v) 由原图像中坐标为 (i,j)、(i+1,j)、(i,j+1)、(i+1,j+1) 的四个像素的值决定，即：  
`f(i+u,j+v) = (1-u)(1-v)f(i,j) + u(1-v)f(i+1,j) + (1-u)vf(i,j+1) + uvf(i+1,j+1)`  
其中`f(i,j)`表示原图像(i,j)处的的像素值

最近邻插值计算量较小，但可能会造成插值生成的图像灰度值不连续，在灰度变化的地方可能出现明显的锯齿状。双线性插值灰度值连续，图像比较平滑。

In [2]:
# 最近邻插值的源码实现
# 1.info 2.空白目标模版 3.每个xy位置的像素值
import cv2
import numpy as np
img = cv2.imread('image0.jpg',1)
imgInfo = img.shape
height = imgInfo[0]
width = imgInfo[1]

dstHeight = int(height/2)
dstWidth = int(width/2)
dstImage = np.zeros((dstHeight,dstWidth,3), np.uint8) #0-255 

for i in range(0, dstHeight): #行
    for j in range(0, dstWidth): #列 
        iNew = round(i * (height*1.0/dstHeight))
        jNew = round(j * (width*1.0/dstWidth))
        dstImage[i, j] = img[iNew, jNew]

cv2.imshow('dst',dstImage)
cv2.waitKey(0)

-1

## 图片剪切

<img src='images/图片剪切.jpg' width=50%>

In [3]:
# x:(100,300)
# y:(100,200)
import cv2
img = cv2.imread('image0.jpg',1)
imgInfo = img.shape
dst = img[100:200, 100:300]
cv2.imshow('image',dst)
cv2.waitKey(0)

-1

## 图片位移

<img src='images/图片位移.jpg' width=50%>

In [4]:
import cv2
import numpy as np
img = cv2.imread('image0.jpg',1)
cv2.imshow('src',img)
imgInfo = img.shape
height = imgInfo[0]
width = imgInfo[1]

# 移位矩阵，水平方向移动100个像素，竖直方向移动200个像素
matShift = np.float32([[1,0,100],[0,1,200]])#  2行3列
dst = cv2.warpAffine(img, matShift, (height,width))#1 data 2 mat 3 info
cv2.imshow('dst', dst)
cv2.waitKey(0)

-1

仿射变换就是图像的线性变换（旋转、缩放）加上平移操作。

[
opencv学习(三十五)之仿射变换warpAffine](https://blog.csdn.net/keith_bb/article/details/56331356)  
[基础——仿射变换矩阵与warpAffine](https://blog.csdn.net/fadefaraway/article/details/54970189)

```python
warpAffine API的实现原理：
[[1,0,100],[0,1,200]] 把该矩阵拆分为如下两个矩阵：
[[1,0],[0,1]]  2*2  A
[[100],[200]]  2*1  B
输入矩阵x,y    2*1  C
矩阵A进行线性变换，矩阵B进行平移
A*C+B = [[1*x+0*y],[0*x+1*y]]+[[100],[200]]
      = [[x+100],[y+200]] # 得到新的x,y，表示变换后的矩阵
```

通常用2x3矩阵来表示仿射变换。原像素点坐标(x,y)，经过仿射变换后的点的坐标是T，则矩阵仿射变换基本算法原理：

<img src='images/仿射变换公式1.png' width=22%>
<img src='images/仿射变换公式2.png' width=12%>

In [5]:
# (10,20)->(110,120)
import cv2
import numpy as np
img = cv2.imread('image0.jpg',1)
cv2.imshow('src',img)
imgInfo = img.shape
height = imgInfo[0]
width = imgInfo[1]
dst = np.zeros(img.shape,np.uint8)

for i in range(0, height-50): # 下移50个像素
    for j in range(0, width-100): # 右移100个像素
        dst[i+50, j+100]=img[i, j]

cv2.imshow('image',dst)
cv2.waitKey(0)

-1

## 图片镜像

<img src='images/图片镜像.jpg' width=50%>

In [7]:
import cv2
import numpy as np
img = cv2.imread('image0.jpg', 1)
cv2.imshow('src', img)
imgInfo = img.shape
height = imgInfo[0]
width = imgInfo[1]
deep = imgInfo[2]
# 新图模板
newImgInfo = (height*2, width, deep)
dst = np.zeros(newImgInfo, np.uint8) # uint8 

for i in range(0, height):
    for j in range(0, width):
        dst[i,j] = img[i,j] # 绘制上半部分
        # (x，2*h - y -1)
        dst[height*2-i-1, j] = img[i,j] # 绘制下半部分
        # (x, y - h) 或 (x, y + h)
        # dst[i-height, j] = img[i,j]

for i in range(0,width): # 添加分隔线
    dst[height,i] = (0,0,255)#BGR
cv2.imshow('dst',dst)
cv2.waitKey(0)

-1

## 图片缩放

利用之前介绍的矩阵移位公式实现图片缩放。
```python
[[A1 A2 B1],[A3 A4 B2]]
[[A1 A2],[A3 A4]]  [[B1],[B2]]
newX = A1*x + A2*y + B1
newY = A3*x + A4*y + B2
比如缩放0.5：x->x*0.5  y->y*0.5
如果把A1,A4设置为0.5，A2,B1,A3设置为0，所以 newX = 0.5*x
```

In [10]:
import cv2
import numpy as np
img = cv2.imread('image0.jpg',1)
cv2.imshow('src',img)
imgInfo = img.shape
height = imgInfo[0]
width = imgInfo[1]
# 缩放矩阵
matScale = np.float32([[0.5,0,0],[0,0.5,0]])
dst = cv2.warpAffine(img, matScale, (int(width/2),int(height/2)))
# dst = cv2.warpAffine(img, matScale, (width,height))
cv2.imshow('dst',dst)
cv2.waitKey(0)

-1

## 仿射变换

通过仿射变换任意变换图形，需要源图像和目标图像上分别一一映射的三个点来定义仿射变换。  
将图片中的每个像素点按照一定的规律映射到新的位置，通过把原图片上三个点（左上角、左下角和右上角）重新映射到新的位置来实现仿射变换。

<img src='images/仿射变换.jpg' width=50%>

In [9]:
import cv2
import numpy as np
img = cv2.imread('image0.jpg',1)
cv2.imshow('src',img)
imgInfo = img.shape
height = imgInfo[0]
width = imgInfo[1]
# src 3->dst 3 (左上角 左下角 右上角)
matSrc = np.float32([[0, 0],[0, height-1],[width-1, 0]])
matDst = np.float32([[50, 50],[300, height-200],[width-300, 100]])
# 组合
matAffine = cv2.getAffineTransform(matSrc, matDst)# 1 src 2 dst
dst = cv2.warpAffine(img, matAffine, (width,height))
cv2.imshow('dst',dst)
cv2.waitKey(0)

-1

## 图片旋转

In [10]:
import cv2
import numpy as np
img = cv2.imread('image0.jpg',1)
cv2.imshow('src',img)
imgInfo = img.shape
height = imgInfo[0]
width = imgInfo[1]
# 旋转矩阵：生成图像旋转所需的仿射变换矩阵，2行3列
matRotate = cv2.getRotationMatrix2D((height*0.5,width*0.5), 45, 0.5)# 1 中心坐标 2 angle 3 scale
#100*100 25 
dst = cv2.warpAffine(img,matRotate,(height,width))
cv2.imshow('dst',dst)
cv2.waitKey(0)

-1